In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{col}

In [2]:
val spark = SparkSession.builder().appName("Movie Title Duplication Removal").master("local[*]").config("spark.jars", "GCS/spark-avro_2.12-3.5.1.jar").getOrCreate()

spark = org.apache.spark.sql.SparkSession@6a5b5a53


org.apache.spark.sql.SparkSession@6a5b5a53

In [3]:
val sprk_ctx = spark.sparkContext

sprk_ctx = org.apache.spark.SparkContext@3747a1bb


org.apache.spark.SparkContext@3747a1bb

In [4]:
%AddJar https://repo1.maven.org/maven2/org/apache/spark/spark-avro_2.12/3.5.1/spark-avro_2.12-3.5.1.jar

Starting download from https://repo1.maven.org/maven2/org/apache/spark/spark-avro_2.12/3.5.1/spark-avro_2.12-3.5.1.jar
Finished download of spark-avro_2.12-3.5.1.jar


Waiting for a Spark session to start...

Using cached version of spark-avro_2.12-3.5.1.jar


In [5]:
val movie_gcs_path = "gs://artifacts_spark/movie.csv"

val movie_df = spark.read.format("csv").option("header", true).option("inferSchema", true).load(movie_gcs_path)

movie_gcs_path = gs://artifacts_spark/movie.csv
movie_df = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [6]:
val movieDfCount = movie_df.count()

movieDfCount = 27278


27278

In [7]:
val titleDistinctDf = movie_df.select("title").distinct().count()

titleDistinctDf = 27262


27262

In [8]:
val modieId = movie_df.select("movieId").distinct().count()

modieId = 27278


27278

In [9]:
movie_df.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [10]:
val dataRdd = movie_df.rdd.map(row => (row.getInt(0), row.getString(1)))

dataRdd = MapPartitionsRDD[46] at map at <console>:43


MapPartitionsRDD[46] at map at <console>:43

In [11]:
val duplicatesRm = dataRdd.distinct().toDF("movieId", "processed_title")

duplicatesRm = [movieId: int, processed_title: string]


[movieId: int, processed_title: string]

In [12]:
val joinedDf = movie_df.join(duplicatesRm, "movieId")

joinedDf = [movieId: int, title: string ... 2 more fields]


[movieId: int, title: string ... 2 more fields]

In [13]:
val cleanedDf = joinedDf.select("movieId", "processed_title", "genres")

cleanedDf = [movieId: int, processed_title: string ... 1 more field]


[movieId: int, processed_title: string ... 1 more field]

In [14]:
cleanedDf.write.mode("overwrite").format("avro").save("hdfs:///user/mano/movie_cleaned_data/")

In [15]:
spark.stop()